In [2]:
import whisper
import torch
import sys
sys.path.append('/opt/ml/input/code/final-project-level3-nlp-12/riffusion')
import os
from typing import Iterator, TextIO
from transformers import pipeline
from _interpolation import Riffusion_interpolation
import re

In [3]:
text = """
[00:00.000 --> 00:06.880]  I heard that Woo-King taught Lee Kyung-kyu, a 42-year-old entertainer, how to broadcast.
[00:06.880 --> 00:11.240]  Of course, I'm sure you've done a lot of studio programs like this.
[00:11.240 --> 00:16.120]  In the Internet live broadcast, the real-time communication is a little different.
[00:16.120 --> 00:18.040]  Communication is really important.
[00:18.040 --> 00:20.200]  He's always looking at the camera.
[00:20.200 --> 00:23.760]  Actually, in the Internet live broadcast, you have to look at this more than this.
[00:23.760 --> 00:26.120]  That's right. The comments are hard.
[00:26.120 --> 00:30.720]  And in the live broadcast, there's a donation system.
[00:30.720 --> 00:32.920]  That's a no-no.
[00:32.920 --> 00:35.520]  You have to listen to it no matter what.
[00:35.520 --> 00:36.920]  Oh, if you listen to the sponsorship, you have to do this.
[00:36.920 --> 00:39.080]  Thank you for your donation.
[00:39.080 --> 00:40.640]  You don't have to do it regardless of the program.
[00:40.640 --> 00:43.160]  Oh, I'll leave it alone.
[00:43.160 --> 00:45.040]  It's good to get a lot of pay.
[00:45.040 --> 00:47.360]  That's the same thing.
[00:47.360 --> 00:51.000]  There are real-time comments, so there's a tip for reading them.
[00:51.000 --> 00:52.280]  There's going to be a lot of comments.
[00:52.280 --> 00:53.200]  It's been a while.
[00:53.200 --> 00:56.160]  Almost 4,000 to 5,000 people go and watch it.
[00:56.160 --> 00:57.080]  How do I read this?
[00:57.080 --> 01:01.400]  Then you can read people like Kim Gu-ran.
[01:01.400 --> 01:06.320]  Like Kim Gu-ran, there are people who always talk in the 12th group in the comments.
[01:06.320 --> 01:09.680]  I don't leave comments.
[01:09.680 --> 01:11.680]  That's the way you talk.
[01:11.680 --> 01:13.200]  I don't leave comments on purpose.
[01:13.200 --> 01:15.200]  I don't leave comments on purpose, everyone.
[01:15.200 --> 01:16.200]  There are people who scratch it.
[01:16.200 --> 01:17.200]  I scratch it for fun.
[01:17.200 --> 01:19.200]  Then it becomes a tiki-taka.
[01:19.200 --> 01:22.200]  If you focus on those people, you can have fun on the air.
[01:22.200 --> 01:24.200]  But it went by so fast.
[01:24.200 --> 01:25.200]  How do you catch it?
[01:25.200 --> 01:26.200]  It's good for your eyesight.
[01:26.200 --> 01:27.200]  That's right.
[01:27.200 --> 01:30.200]  No, we're just talking about eye sight.
[01:30.200 --> 01:34.200]  I'm the only one who sees it on the screen.
[01:34.200 --> 01:35.200]  It's fast.
[01:35.200 --> 01:36.200]  It's fast.
[01:36.200 --> 01:39.200]  The problem is that Won is here.
[01:39.200 --> 01:45.200]  I can't see it, so I keep looking at it like this.
[01:45.200 --> 01:50.200]  Okay, there's a special skill that makes you excited when you do that.
[01:50.200 --> 01:51.200]  There should be one.
[01:51.200 --> 01:52.200]  This, this.
[01:52.200 --> 01:53.200]  That's right.
[01:53.200 --> 01:54.200]  Actually, I didn't make it now.
[01:54.200 --> 01:58.200]  What's sitting here right now is my knee dance.
[01:58.200 --> 01:59.200]  This is a real signature dance.
[01:59.200 --> 02:00.200]  This is what you're looking at.
[02:00.200 --> 02:03.200]  Didn't you say you're going to pay for the donation?
[02:03.200 --> 02:04.200]  Depending on the donation amount.
[02:04.200 --> 02:05.200]  Yes.
[02:05.200 --> 02:06.200]  This is the highest price.
[02:06.200 --> 02:07.200]  The highest price.
[02:07.200 --> 02:10.200]  Let's take a look.
[02:10.200 --> 02:13.200]  I can't believe you're looking at this.
[02:13.200 --> 02:23.200]  Let's go with 70 million won.
[02:27.200 --> 02:28.200]  It's 1 million won.
[02:28.200 --> 02:29.200]  It's 1 million won.
[02:29.200 --> 02:30.200]  It's 1 million won.
[02:30.200 --> 02:31.200]  To the side.
[02:31.200 --> 02:31.200]  To the side.
[02:31.200 --> 02:41.200]  To the side.
[02:41.200 --> 02:46.200]  Wow, how does this work?
[02:46.200 --> 02:48.200]  I've never seen this before.
[02:48.200 --> 03:13.200]  It's an honor. It's an honor.
"""

In [58]:
times = re.findall(r'\[.+\]', text)
result = []
for t in times:
            start, end = t.split('-->')
            start, end = int(start[1:].strip()[:-4].split(':')[0])*60+int(start[1:].strip()[:-4].split(':')[1]), int(end[1:].strip()[:-4].split(':')[0])*60+int(end[1:].strip()[:-5].split(':')[1])
            result.append([start, end])
print(result)

[[0, 6], [6, 11], [11, 16], [16, 18], [18, 20], [20, 23], [23, 26], [26, 30], [30, 32], [32, 35], [35, 36], [36, 39], [39, 40], [40, 43], [43, 45], [45, 47], [47, 51], [51, 52], [52, 53], [53, 56], [56, 57], [57, 61], [61, 66], [66, 69], [69, 71], [71, 73], [73, 75], [75, 76], [76, 77], [77, 79], [79, 82], [82, 84], [84, 85], [85, 86], [86, 87], [87, 90], [90, 94], [94, 95], [95, 96], [96, 99], [99, 105], [105, 110], [110, 111], [111, 112], [112, 113], [113, 114], [114, 118], [118, 119], [119, 120], [120, 123], [123, 124], [124, 125], [125, 126], [126, 127], [127, 130], [130, 133], [133, 143], [147, 148], [148, 149], [149, 150], [150, 151], [151, 151], [151, 161], [161, 166], [166, 168], [168, 193]]


In [37]:
text = re.sub(r'\[.+\]', '', text).split('\n')
text = [t.strip() for t in text if t]

In [15]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment = pipeline("sentiment-analysis", model='cardiffnlp/twitter-roberta-base-sentiment-latest', tokenizer='cardiffnlp/twitter-roberta-base-sentiment-latest')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
sentiments = []
for t in text:
    sentiments.append(sentiment(t))
flag = None
contents = []
for x,y in zip(text, sentiments):
    if flag == None:
        if y[0]['label'] == 'positive':
            flag = True
            contents.append([x])
        elif y[0]['label'] == 'negative':
            flag = False
            contents.append([x])
    else:
        if flag == False:
            if y[0]['label'] == 'positive':
                flag = True
                contents.append([x])
            else:
                contents[-1] += [x]
        else:
            if y[0]['label'] == 'negative':
                flag = False
                contents.append([x])
            else:
                contents[-1] += [x]
contents = list(map(''.join, contents))
# summarized = []
# for x in contents:
#     summarized.append(summarizer(x))
contents

Your max_length is set to 142, but you input_length is only 67. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 142, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 142, but you input_length is only 25. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 142, but you input_length is only 31. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 142, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 142, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 142, but you input_length is only 64. You might consider

[[{'summary_text': "In the Internet live broadcast, the real-time communication is a little different. Of course, I'm sure you've done a lot of studio programs like this. He's always looking at the camera.Actually, in the Internet Live broadcast, you have to look at this more."}],
 [{'summary_text': "You have to listen to it no matter what. That's right. The comments are hard. And in the live broadcast, there's a donation system.That's a no-no. Oh, if you listen to the sponsorship, you have to do this. That is right."}],
 [{'summary_text': "There are real-time comments, so there's a tip for reading them. Almost 4,000 to 5,000 people go and watch it. You don't have to do it regardless of the program. It's good to get a lot of pay. That's the same thing."}],
 [{'summary_text': "I can't see it, so I keep looking at it like this. The problem is that Won is here. I can't seem to see it. I'm looking for it, but it's not there. I don't know what it is. I just want to know what's going on."}],

In [53]:
import whisper
import torch
import sys
sys.path.append('/opt/ml/input/code/final-project-level3-nlp-12/riffusion')
import os
from typing import Iterator, TextIO
from transformers import pipeline
from interpolation import Riffusion_interpolation
import re

class SentimentModel():
    def __init__(self, text):
        self.text = self.preprocessing(text)
        self.time = self.gettime(text)
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        #self.sentiment = pipeline("sentiment-analysis", model='cardiffnlp/twitter-roberta-base-sentiment-latest', tokenizer='cardiffnlp/twitter-roberta-base-sentiment-latest')
        self.classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
    
    def preprocessing(self, text):
        text = re.sub(r'\[.+\]', '', text).split('\n')
        text = [t.strip() for t in text if t]
        return text
    
    def gettime(self, text):
        times = re.findall(r'\[.+\]', text)
        result = []
        for t in times:
            start, end = t.split('-->')
            start, end = int(start[1:].strip()[:-4].split(':')[0])*60+int(start[1:].strip()[:-4].split(':')[1]), int(end[1:].strip()[:-4].split(':')[0])*60+int(end[1:].strip()[:-5].split(':')[1])
            result.append([start, end])
        return result
    # def run(self):
    #     sentiments = []
    #     text = self.text
    #     for t in text:
    #         sentiments.append(self.sentiment(t))
    #     flag = None
    #     contents = []
    #     for x,y in zip(text, sentiments):
    #         if flag == None:
    #             if y[0]['label'] == 'positive':
    #                 flag = True
    #                 contents.append([x])
    #             elif y[0]['label'] == 'negative':
    #                 flag = False
    #                 contents.append([x])
    #         else:
    #             if flag == False:
    #                 if y[0]['label'] == 'positive':
    #                     flag = True
    #                     contents.append([x])
    #                 else:
    #                     contents[-1] += [x]
    #             else:
    #                 if y[0]['label'] == 'negative':
    #                     flag = False
    #                     contents.append([x])
    #                 else:
    #                     contents[-1] += [x]
    #     contents = list(map(''.join, contents))
    #     summarized = []
    #     for x in contents:
    #         summarized.append(self.summarizer(x))
    #     return summarized
    def run(self):
        sentiments = []
        text = self.text
        for t in text:
            result = self.classifier(t)[0]
            result.sort(key = lambda x: x['score'], reverse = True)
            sentiments.append(result[0]['label'])
        sentiment = None
        contents = []
        threshold = 2
        start = 0
        for x, y in zip(sentiments, self.time):
            if sentiment == None:
                if x != 'neutral':
                    sentiment = x
                    start = y[0]
            else:
                if x != sentiment and x != 'neutral':
                    if y[0] - start >= threshold:
                        contents.append([start, y[0], sentiment])
                    sentiment = x
                    start = y[0]
        return contents

In [54]:
sen = SentimentModel(text)
summarized_content = sen.run()
summarized_content

[[36, 56, 'joy'], [56, 79, 'surprise'], [79, 82, 'joy'], [82, 168, 'surprise']]

In [12]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
result = classifier("I love this!")[0]
result.sort(key = lambda x: x['score'], reverse = True)

/opt/conda/envs/whisper/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [31]:
def gettime(text):
        times = re.findall(r'\[.+\]', text)
        result = []
        for t in times:
            start, end = t.split('-->')
            start, end = start[1:].strip()[:-4], end[:-1].strip()[:-4]
            result.append([start, end])
        return result

In [32]:
time = gettime(text)
time

[['00:00', '00:06'],
 ['00:06', '00:11'],
 ['00:11', '00:16'],
 ['00:16', '00:18'],
 ['00:18', '00:20'],
 ['00:20', '00:23'],
 ['00:23', '00:26'],
 ['00:26', '00:30'],
 ['00:30', '00:32'],
 ['00:32', '00:35'],
 ['00:35', '00:36'],
 ['00:36', '00:39'],
 ['00:39', '00:40'],
 ['00:40', '00:43'],
 ['00:43', '00:45'],
 ['00:45', '00:47'],
 ['00:47', '00:51'],
 ['00:51', '00:52'],
 ['00:52', '00:53'],
 ['00:53', '00:56'],
 ['00:56', '00:57'],
 ['00:57', '01:01'],
 ['01:01', '01:06'],
 ['01:06', '01:09'],
 ['01:09', '01:11'],
 ['01:11', '01:13'],
 ['01:13', '01:15'],
 ['01:15', '01:16'],
 ['01:16', '01:17'],
 ['01:17', '01:19'],
 ['01:19', '01:22'],
 ['01:22', '01:24'],
 ['01:24', '01:25'],
 ['01:25', '01:26'],
 ['01:26', '01:27'],
 ['01:27', '01:30'],
 ['01:30', '01:34'],
 ['01:34', '01:35'],
 ['01:35', '01:36'],
 ['01:36', '01:39'],
 ['01:39', '01:45'],
 ['01:45', '01:50'],
 ['01:50', '01:51'],
 ['01:51', '01:52'],
 ['01:52', '01:53'],
 ['01:53', '01:54'],
 ['01:54', '01:58'],
 ['01:58', '0

In [55]:
times = re.findall(r'\[.+\]', text)
timeline = []
for t in times:
    start, end = t.split('-->')
    start, end = int(start[1:].strip()[:-4].split(':')[0])*60+int(start[1:].strip()[:-4].split(':')[1]), int(end[1:].strip()[:-4].split(':')[0])*60+int(end[1:].strip()[:-5].split(':')[1])
    timeline.append([start, end])

0 6
6 11
11 16
16 18
18 20
20 23
23 26
26 30
30 32
32 35
35 36
36 39
39 40
40 43
43 45
45 47
47 51
51 52
52 53
53 56
56 57
57 61
61 66
66 69
69 71
71 73
73 75
75 76
76 77
77 79
79 82
82 84
84 85
85 86
86 87
87 90
90 94
94 95
95 96
96 99
99 105
105 110
110 111
111 112
112 113
113 114
114 118
118 119
119 120
120 123
123 124
124 125
125 126
126 127
127 130
130 133
133 143
147 148
148 149
149 150
150 151
151 151
151 161
161 166
166 168
168 193


In [1]:
class SentimentModel():
    def __init__(self, text, timeline):
        self.time = timeline
        self.text = self.preprocessing(text)
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        self.classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
    
    def preprocessing(self, text):
        text = re.sub(r'\[.+\]', '', text).split('\n')
        text = [t.strip() for t in text if t]
        return text
    
    def run(self):
        sentiments = []
        text = self.text
        for t in text:
            result = self.classifier(t)[0]
            result.sort(key = lambda x: x['score'], reverse = True)
            sentiments.append(result[0]['label'])
        sentiment = None
        contents = []
        min = 5
        max = 20
        start = 0
        for x, y in zip(sentiments, self.time):
            if sentiment == None:
                if x != 'neutral':
                    contents.append([start, int(y[0]), None])
                    sentiment = x
                    start = y[0]
            else:
                if x != sentiment and x != 'neutral':
                    if int(y[0]) - start > min and int(y[0]) - start < max:
                        contents.append([start, int(y[0]), sentiment])
                    else:
                        contents.append([start, int(y[0]), None])
                    sentiment = x
                    start = int(y[0])
        # contents = [[3, 5, 'angry']] -> [3, [3, 5, 'angry'], 32, [37, 42, 'surprise']]
        return contents

In [ ]:
sent = SentimentModel(, )
result = sent.run()